In this notebook, we are going to try to improve a given solution of an optimization problem, `Golomb ruler` of size 11.
First, we load the snapshot version of choco (this version is not released at the time this notebook is written but contains helpful features).

In [ ]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-snapshots</id>
  <url>https://oss.sonatype.org/content/repositories/snapshots/</url>
</repository>
<dependency>
  <groupId>org.choco-solver</groupId>
  <artifactId>choco-solver</artifactId>
  <version>4.0.9</version>
</dependency>

Now, let's import classes required for modeling.

In [ ]:
import org.chocosolver.solver.Model;
import org.chocosolver.solver.variables.IntVar;

In [ ]:
int m = 11;
Model model = new Model();
IntVar[] ticks = model.intVarArray("a", m, 0, (m < 31) ? (1 << (m + 1)) - 1 : 9999, false);
IntVar[] diffs = model.intVarArray("d", (m * m - m) / 2, 0, (m < 31) ? (1 << (m + 1)) - 1 : 9999, false);
model.arithm(ticks[0], "=", 0).post();

for (int i = 0; i < m - 1; i++) {
    model.arithm(ticks[i + 1], ">", ticks[i]).post();
}

for (int k = 0, i = 0; i < m - 1; i++) {
    for (int j = i + 1; j < m; j++, k++) {
        // d[k] is m[j]-m[i] and must be at least sum of first j-i integers
        model.arithm(ticks[j], "-", ticks[i], "=", diffs[k]).post();
        model.arithm(diffs[k], ">=", (j - i) * (j - i + 1) / 2).post();
        model.arithm(diffs[k], "-", ticks[m - 1], "<=", -((m - 1 - j + i) * (m - j + i)) / 2).post();
        model.arithm(diffs[k], "<=", ticks[m - 1], "-", ((m - 1 - j + i) * (m - j + i)) / 2).post();
    }
}
model.allDifferent(diffs, "BC").post();
// break symetries
if (m > 2) {
    model.arithm(diffs[0], "<", diffs[diffs.length - 1]).post();
}
model.setObjective(Model.MINIMIZE,ticks[m - 1]);

The model is now declared and ready to be solved.
Here, we are considering the following case: we have pre-computed a good quality solution `good`.
So, we want to find an improving solution *close* to the given one.

In [ ]:
int[] good = new int[]{0, 1, 7, 25, 28, 33, 47, 63, 67, 76, 78};
ticks[m - 1].lt(78).post(); // a better solution must be find

There are three options:
1. using a Large Neighborhood Search strategy,
2. using a Limited-Discrepancy Search strategy and
3. using a solution-based phase saving approach.

## Using LNS
Large Neighborhood Search mimics a local search algorithm: it freezes some variables to their value in the last solution and let the others being fixed by the search strategy. 
Ususally, the user let the solver finds the first solution. But, it is also possible to load a solution to start from.

The first step is to load the solution into a `Solution` object.

In [ ]:
import org.chocosolver.solver.Solution;
import java.util.stream.IntStream;

Solution sol = new Solution(model, ticks);
IntStream.range(0, m).forEach(i -> sol.setIntVal(ticks[i], good[i]));

Then, a LNS can be declared:

In [ ]:
import org.chocosolver.solver.search.limits.FailCounter;
import org.chocosolver.solver.search.loop.lns.INeighborFactory;
import org.chocosolver.solver.search.loop.lns.neighbors.INeighbor;
import org.chocosolver.solver.search.loop.move.MoveLNS;


MoveLNS lns = new MoveLNS(
    model.getSolver().getMove(), // this represents the default way to explore the search, ie DFS 
    INeighborFactory.random(ticks), // this represents the way neighborhood will be generated, here randomly
    new FailCounter(model, 100) // this represents when to try a new neighborhood, here every 100 failures
);  
lns.loadFromSolution(sol, model.getSolver()); // the solution is then loaded
model.getSolver().setMove(lns); // an LNS is declared

Now, the search can start:

In [ ]:
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s"); // to avoid never ending loop
model.getSolver().solve(); // look for the first improving solution

## Using LDS
Limited-Discrepancy Search bets that the enumeration strategy (variable selection + value selection) is quite enough.
So it tries not to move too much away from it (see "W.D. Harvey and M.L.Ginsberg, *Limited Discrepancy Search*, IJCAI-95" for more details).
This approach requires to declare an enumeration strategy based on the solution.

In [ ]:
import org.chocosolver.solver.search.strategy.Search;
import org.chocosolver.solver.search.strategy.selectors.variables.InputOrder;

import java.util.Arrays;
import java.util.Map;
import java.util.stream.Collectors;

model.getSolver().hardReset(); // required for notebook 

Map<IntVar, Integer> map = IntStream.range(0,m).boxed().collect(Collectors.toMap(i->ticks[i], i -> good[i]));
model.getSolver().setSearch(Search.intVarSearch(
        new InputOrder<>(model), // variable selection: from ticks[0] to ticks[m-1]
        var -> { // value selection, choose value from solution if possible
            if(var.contains(map.get(var))){
                return map.get(var);
            }
            return var.getLB(); // otherwise, select the current lower bound
        },
        ticks
));
model.getSolver().setLDS(12); // discrepancy is set to 12
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s");
model.getSolver().solve();

## Solution-based phase saving approach
This approach is a kind of compromise between LNS and LDS. It requires a solution to be applied (most of the time, found by the solver) and then when a value selection is called, it tries first the value in the last solution found (for more details, see "E. Demirović, G. Chu and P.J. Stuckey, *Solution-based phase saving for CP: A value-selection heuristic to simulate local search behavior in complete solvers*, CP18").

In [ ]:
import org.chocosolver.solver.search.strategy.Search;
import org.chocosolver.solver.search.strategy.selectors.variables.InputOrder;
import org.chocosolver.solver.search.strategy.selectors.values.IntDomainLast;
import org.chocosolver.solver.search.strategy.selectors.values.IntDomainMin;

model.getSolver().hardReset(); // required for notebook

model.getSolver().setSearch(Search.intVarSearch(
    new InputOrder<>(model), // variable selection: from ticks[0] to ticks[m-1]
    new IntDomainLast(sol, new IntDomainMin()), // value selection, and default one if value from solution is forbidden
    ticks
));
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s");
model.getSolver().solve();